In [ ]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

import copy

import torch
import socialforce

_ = torch.manual_seed(45)

(pedped-1d-bayesian)=
# 1D Bayesian

This is a Bayesian version of {ref}`pedped-1d`.

In [ ]:
V1 = socialforce.potentials.PedPedPotential(sigma=0.3)
V2 = socialforce.potentials.PedPedPotential(sigma=0.5)
with socialforce.show.canvas(figsize=(12, 8), ncols=2, nrows=2) as ((ax1, ax2), (ax3, ax4)):
    socialforce.show.potential_2d(V1, ax1)
    socialforce.show.potential_2d_grad(V1, ax2)

    socialforce.show.potential_2d(V2, ax3)
    socialforce.show.potential_2d_grad(V2, ax4)

The pedestrian is located in the left focal point of the ellipse and at their 
current speed can reach the right focal point within one step that is assumed
to take $\Delta t = 0.4s$.



## Scenario

We generate 10 {ref}`Circle scenario <scenarios>`, five for each configuration
of the potential.

In [ ]:
scenario1 = socialforce.scenarios.Circle(ped_ped=V1).generate(5)
scenario2 = socialforce.scenarios.Circle(ped_ped=V2).generate(5)
true_experience = socialforce.Trainer.scenes_to_experience(scenario1 + scenario2)


In [ ]:
# HIDE CODE
with socialforce.show.track_canvas(figsize=(8, 5), ncols=2) as (ax1, ax2):
    socialforce.show.states(ax1, scenario1[0], zorder=10)
    for scenario in scenario1[1:]:
        socialforce.show.states(ax1, scenario, alpha=0.5)

    socialforce.show.states(ax2, scenario2[0], zorder=10)
    for scenario in scenario2[1:]:
        socialforce.show.states(ax2, scenario, alpha=0.5)


## Deterministic

In [ ]:
V = socialforce.potentials.PedPedPotentialMLP()
initial_state_dict = copy.deepcopy(V.state_dict())

In [ ]:
# HIDE OUTPUT
simulator = socialforce.Simulator(ped_ped=V) 
opt = torch.optim.SGD(V.parameters(), lr=1.0)
socialforce.Trainer(simulator, opt).loop(10, true_experience)
final_state_dict = copy.deepcopy(V.state_dict())

In [ ]:
# HIDE CODE
with socialforce.show.canvas(ncols=2) as (ax1, ax2):
    socialforce.show.potential_1d_parametric(
        V1, ax1, ax2, 
        label=r'true $V_1 = V_0 e^{-b/\sigma}$', sigma_label=r'true $\sigma$', color='gray')
    socialforce.show.potential_1d_parametric(
        V2, ax1, ax2, 
        label=r'true $V_2 = V_0 e^{-b/\sigma}$', sigma_label=r'true $\sigma$', color='darkgray')

    V.load_state_dict(initial_state_dict)
    socialforce.show.potential_1d(V, ax1, ax2, label=r'initial MLP($b$)', linestyle='dashed', color='C0')

    V.load_state_dict(final_state_dict)
    socialforce.show.potential_1d(V, ax1, ax2, label=r'MLP($b$)', color='C0')

# Bayesian Ensemble

In [ ]:
V = socialforce.potentials.PedPedPotentialMLP(hidden_units=32, dropout_p=0.1)
initial_state_dict = copy.deepcopy(V.state_dict())

In [ ]:
# HIDE OUTPUT
simulator = socialforce.Simulator(ped_ped=V) 
opt = torch.optim.SGD(V.parameters(), lr=0.2)
socialforce.Trainer(simulator, opt).loop(25, true_experience)
final_state_dict = copy.deepcopy(V.state_dict())

In [ ]:
# HIDE CODE
with socialforce.show.canvas(ncols=2) as (ax1, ax2):
    socialforce.show.potential_1d_parametric(
        V1, ax1, ax2, 
        label=r'true $V_1 = V_0 e^{-b/\sigma}$', sigma_label=r'true $\sigma$', color='gray')
    socialforce.show.potential_1d_parametric(
        V2, ax1, ax2, 
        label=r'true $V_2 = V_0 e^{-b/\sigma}$', sigma_label=r'true $\sigma$', color='darkgray')

    # V.load_state_dict(initial_state_dict)
    # socialforce.show.potential_1d(V, ax1, ax2, label=r'initial MLP($b$)', linestyle='dashed', color='C0')

    V.load_state_dict(final_state_dict)
    socialforce.show.potential_1d(V, ax1, ax2, label=r'MLP($b$)', color='C0')

## VAE

In [ ]:
class VAE(torch.nn.Module):
    """Based on the pytorch VAE example."""
    
    def __init__(self, *, predict_dim=1, hidden_units=20, z_dim=2):
        super().__init__()
        self.z_dim = z_dim

        # encoder
        self.fc1 = torch.nn.Linear(predict_dim, hidden_units)
        self.fc21 = torch.nn.Linear(hidden_units, z_dim)
        self.fc22 = torch.nn.Linear(hidden_units, z_dim)
        
        # decoder
        self.fc3 = torch.nn.Linear(z_dim, hidden_units)
        self.fc4 = torch.nn.Linear(hidden_units, hidden_units)
        self.fc5 = torch.nn.Linear(hidden_units, predict_dim)

    def encode(self, x):
        h1 = torch.nn.functional.softplus(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = torch.nn.functional.softplus(self.fc3(z))
        h4 = torch.nn.functional.softplus(self.fc4(h3))
        return self.fc5(h4)

    def forward(self, x):
        if self.training:
            mu, logvar = self.encode(x)
        else:
            mu = torch.zeros((x.shape[0], self.z_dim), requires_grad=False)
            logvar = torch.zeros((x.shape[0], self.z_dim), requires_grad=False)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar
    
vae_model = VAE()
vae_model_init = copy.deepcopy(vae_model.state_dict())

In [ ]:
def vae_loss_l1(y_hat, target, mu, logvar, *, kld_prefactor=1.0):
    recon_loss = torch.nn.functional.l1_loss(y_hat, target, reduction='mean')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp()) / y_hat.shape[0]

    return recon_loss + kld_prefactor * KLD

In [ ]:
vae_model.load_state_dict(vae_model_init)
vae_model.train()
optimizer = torch.optim.SGD(vae_model.parameters(), lr=0.01, momentum=0.9)
for epoch in range(10000):
    optimizer.zero_grad()
    y_hat, mu, logvar = vae_model(target)
    loss = vae_loss_l1(y_hat, target, mu, logvar, kld_prefactor=2.0)
    loss.backward()
    optimizer.step()